In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


# Importing and Data Preprocessing

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [4]:
train['Age'].fillna(train['Age'].mean(), inplace = True)
test['Age'].fillna(train['Age'].mean(), inplace = True)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace = True)
test['Embarked'].fillna(train['Embarked'].mode()[0], inplace = True)
train['Fare'].fillna(train['Fare'].mode().iloc[0], inplace = True)
test['Fare'].fillna(train['Fare'].mode().iloc[0], inplace = True)

In [5]:
le = LabelEncoder()

train['Sex'] = le.fit_transform(train['Sex'].values)
train['Embarked'] = le.fit_transform(train['Embarked'].values)


test['Sex'] = le.fit_transform(test['Sex'].values)
test['Embarked'] = le.fit_transform(test['Embarked'].values)

In [6]:
COLUMNS_TO_DROP = ['PassengerId', 'Name', 'Cabin', 'Ticket']
pIds = test['PassengerId']
train.drop(COLUMNS_TO_DROP, axis=1, inplace=True)
test.drop(COLUMNS_TO_DROP, axis=1, inplace=True)

In [7]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [8]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [9]:
y = train['Survived']
x = train.drop('Survived', axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 46)

# Gradient Boost Classifier

In [ ]:
# Using A GradientBoostClassifier

grid_values = {'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'n_estimators': [1, 10, 50, 100, 1000], 'max_features' : ['auto', 'sqrt', 'log2']}

clf = GridSearchCV(GradientBoostingClassifier(random_state = 0), param_grid = grid_values, scoring = 'accuracy')
clf.fit(X_train, y_train)
report = classification_report(y_test, clf.predict(X_test))
print(report)

In [ ]:
clf.best_estimator_

# Logistic Regression

In [ ]:
# Using a LogisticRegression Model

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_LR = scaler.fit_transform(X_train)
X_test_LR = scaler.transform(X_test)

grid_values = {'C' : [0.1, 1, 10, 100], 'max_iter': [10000]}

clfLR = GridSearchCV(LogisticRegression(random_state = 0), param_grid = grid_values, scoring = 'accuracy')
clfLR.fit(X_train_LR, y_train)
report = classification_report(np.array(y_test).reshape(-1,1), clfLR.predict(X_test_LR))
print(report)

In [ ]:
clfLR.best_estimator_

# Support Vector Machine

In [ ]:
# Using SVM

from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_SVM = scaler.fit_transform(X_train)
X_test_SVM = scaler.transform(X_test)

grid_values = {'C' : [0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree' : [1,2,3], 'gamma' : ['scale', 'auto']}

clfSVM = GridSearchCV(SVC(random_state = 0), param_grid = grid_values, scoring = 'accuracy')
clfSVM.fit(X_train_SVM, y_train)
report = classification_report(y_test, clfSVM.predict(X_test_SVM))
print(report)

In [ ]:
clfSVM.best_estimator_

# Naive Bayes Classifier 

In [ ]:
# Naive Bayes Classifier

from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_GNB = scaler.fit_transform(X_train)
X_test_GNB = scaler.transform(X_test)

clfGNB = GaussianNB()
clfGNB.fit(X_train_GNB, y_train)
report = classification_report(y_test, clfGNB.predict(X_test_GNB))
print(report)

# Dummy Classifier for Benchmark

In [ ]:
# Using Dummy Classifier

from sklearn.dummy import DummyClassifier

grid_values = {'strategy' : ['stratified', 'most_frequent', 'prior', 'uniform']}

clfSVM = GridSearchCV(DummyClassifier(random_state = 0), param_grid = grid_values, scoring = 'accuracy')
clfSVM.fit(X_train, y_train)
report = classification_report(y_test, clfSVM.predict(X_test))
print(report)

# Random Forest Classifier

In [ ]:
# Using A RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier

grid_values = {'n_estimators': [1, 10, 50, 100, 1000], 'max_depth' : [3,5,7,9,11]}

clfRF = GridSearchCV(RandomForestClassifier(random_state = 0), param_grid = grid_values, scoring = 'accuracy')
clfRF.fit(X_train, y_train)
report = classification_report(y_test, clfRF.predict(X_test))
print(report)

In [ ]:
clfRF.best_estimator_

# Computing Results

In [11]:
clf_ans = GradientBoostingClassifier(learning_rate=0.01, max_features='auto',n_estimators=1000, random_state=0)
clf_ans.fit(x, y)
pd.DataFrame({'PassengerId': pIds, 'Survived': clf_ans.predict(test)})

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [14]:
res = pd.DataFrame({'PassengerId': pIds, 'Survived': clf_ans.predict(test)}).to_csv(r'titanic_submission.csv', index = False)